In [ ]:
# -----------------------------------------------------------------------------
# 1. IMPORTS AND INITIALIZATION
# -----------------------------------------------------------------------------
import ee
import geemap
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import datetime

# Initialize the Earth Engine library.
try:
    geemap.ee_initialize()
except Exception as e:
    print(f"Earth Engine initialization failed. Please authenticate. Error: {e}")
    ee.Authenticate()
    geemap.ee_initialize()

# -----------------------------------------------------------------------------
# 2. GEE ASSET LOADING AND CONSTANTS
# -----------------------------------------------------------------------------
# Load Florida boundaries from the TIGER dataset.
FLORIDA_COUNTIES = ee.FeatureCollection("TIGER/2018/Counties").filter(ee.Filter.eq('STATEFP', '12'))
FLORIDA_STATE = FLORIDA_COUNTIES.union()

# Get county names for the dropdown menu.
try:
    county_names = FLORIDA_COUNTIES.aggregate_array('NAME').getInfo()
    county_names.sort()
    GEOMETRY_OPTIONS = ['Florida'] + county_names
except Exception as e:
    print(f"Could not fetch county names from GEE. Defaulting to Florida only. Error: {e}")
    GEOMETRY_OPTIONS = ['Florida']

# Define band names for Landsat missions for NDVI and EVI calculations.
BAND_INFO = {
    'L8_9': {'NIR': 'SR_B5', 'Red': 'SR_B4', 'Blue': 'SR_B2'},  # Landsat 8 & 9
    'L5_7': {'NIR': 'SR_B4', 'Red': 'SR_B3', 'Blue': 'SR_B1'}   # Landsat 5 & 7
}

# Visualization parameters for NDVI and EVI.
VIS_PARAMS = {
    'NDVI': {
        'min': -1.0, 'max': 1.0,
        'palette': ['beige', 'darkgreen'],
        'label': 'NDVI',
        'rank_high': '🌳 Top 5 Highest (Densest Vegetation)',
        'rank_low': '🍂 Top 5 Lowest (Least Vegetation)'
    },
    'EVI': {
        'min': -1.0, 'max': 1.0,
        'palette': ['#E6E6FA', '#006400'], # Light purple to dark green
        'label': 'EVI',
        'rank_high': '🌳 Top 5 Highest (Healthiest Vegetation)',
        'rank_low': '🍂 Top 5 Lowest (Stressed/No Vegetation)'
    }
}

# -----------------------------------------------------------------------------
# 3. CORE GEE PROCESSING FUNCTIONS
# -----------------------------------------------------------------------------
def mask_landsat_clouds(image):
    """Masks clouds and cloud shadows in Landsat Collection 2 images."""
    qa = image.select('QA_PIXEL')
    cloud_mask = (1 << 3) | (1 << 5)
    mask = qa.bitwiseAnd(cloud_mask).eq(0)
    return image.updateMask(mask)

def apply_scale_factors(image):
    """Applies scaling factors to optical (Surface Reflectance) bands."""
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    return image.addBands(optical_bands, overwrite=True)

def add_vegetation_indices(image):
    """Calculates NDVI and EVI and adds them as bands to the image."""
    scaled_image = apply_scale_factors(image)
    spacecraft = image.get('SPACECRAFT_ID')

    # Select bands based on the satellite
    bands = ee.Dictionary(ee.Algorithms.If(
        ee.List(['LANDSAT_8', 'LANDSAT_9']).contains(spacecraft),
        BAND_INFO['L8_9'],
        BAND_INFO['L5_7']
    ))

    # Calculate NDVI: (NIR - Red) / (NIR + Red)
    ndvi = scaled_image.normalizedDifference([bands.getString('NIR'), bands.getString('Red')]).rename('NDVI')

    # Calculate EVI: 2.5 * ((NIR - Red) / (NIR + 6 * Red - 7.5 * Blue + 1))
    evi = scaled_image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
            'NIR': scaled_image.select(bands.getString('NIR')),
            'RED': scaled_image.select(bands.getString('Red')),
            'BLUE': scaled_image.select(bands.getString('Blue'))
        }).rename('EVI')

    return image.addBands([ndvi, evi])

def get_mean_image_for_year(year, months, geometry, index_name):
    """Creates a mean composite image for a given index, year, month range, and geometry."""
    start_date = ee.Date.fromYMD(year, months[0], 1)
    end_date = ee.Date.fromYMD(year, months[1], 1).advance(1, 'month').advance(-1, 'day')
    
    landsat_collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').merge(
        ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')).merge(
        ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')).merge(
        ee.ImageCollection('LANDSAT/LT05/C02/T1_L2'))

    image_composite = (landsat_collection
                       .filterBounds(geometry)
                       .filterDate(start_date, end_date)
                       .map(mask_landsat_clouds)
                       .map(add_vegetation_indices) # Calculate both indices
                       .select(index_name)         # Select the one the user wants
                       .mean())
    
    return image_composite.set('year', year)

def get_yearly_stats_fc(year, months, index_name, collection, scale):
    """Calculates zonal statistics for a given year and returns a FeatureCollection."""
    mean_image = get_mean_image_for_year(year, months, collection.geometry(), index_name)
    
    return mean_image.reduceRegions(
        collection=collection,
        reducer=ee.Reducer.mean(),
        scale=scale
    ).map(lambda f: f.set('year', year))

# -----------------------------------------------------------------------------
# 4. UI WIDGETS DEFINITION
# -----------------------------------------------------------------------------
header = widgets.HTML("<h2>Florida Vegetation Index Dashboard</h2>")
index_dropdown = widgets.Dropdown(options=['NDVI', 'EVI'], value='NDVI', description='Index:')
geometry_dropdown = widgets.Dropdown(options=GEOMETRY_OPTIONS, description='Area:')

current_year = datetime.datetime.now().year
start_year_slider = widgets.IntSlider(value=2000, min=1984, max=current_year, description='Start Year:')
end_year_slider = widgets.IntSlider(value=current_year, min=1984, max=current_year, description='End Year:')
delta_slider = widgets.IntSlider(value=5, min=1, max=10, description='Delta (yrs):')
month_slider = widgets.IntRangeSlider(value=[4, 9], min=1, max=12, step=1, description='Months:')
run_button = widgets.Button(description="Run Analysis", button_style='success', icon='cogs')
status_label = widgets.Label(value="Status: Ready")
map_output = widgets.Output(layout={'height': '600px'})
table_output = widgets.Output()
rank_output = widgets.Output()

# -----------------------------------------------------------------------------
# 5. UI LAYOUT
# -----------------------------------------------------------------------------
controls_box = widgets.VBox([
    widgets.HTML("<b>1. Select Index & Area</b>"),
    index_dropdown, geometry_dropdown,
    widgets.HTML("<hr><b>2. Define Timeframe</b>"),
    start_year_slider, end_year_slider, delta_slider, month_slider,
    widgets.HTML("<hr>"),
    run_button, status_label
])
output_tabs = widgets.Tab(children=[table_output, rank_output])
output_tabs.set_title(0, 'Statistics Table')
output_tabs.set_title(1, 'County Rankings')
left_panel = widgets.VBox([controls_box, output_tabs], layout=widgets.Layout(width='35%', padding='10px'))
right_panel = widgets.VBox([map_output], layout=widgets.Layout(width='65%'))
ui_layout = widgets.HBox([left_panel, right_panel])

# -----------------------------------------------------------------------------
# 6. EVENT HANDLER (THE "BRAIN")
# -----------------------------------------------------------------------------
def run_analysis(b):
    """Triggered when 'Run Analysis' is clicked."""
    with map_output: clear_output(wait=True)
    with table_output: clear_output(wait=True)
    with rank_output: clear_output(wait=True)
    status_label.value = "Status: Initializing..."

    try:
        # 1. Get user inputs
        selected_index = index_dropdown.value
        selected_geo_name = geometry_dropdown.value
        start_year, end_year = start_year_slider.value, end_year_slider.value
        delta, month_range = delta_slider.value, month_slider.value
        vis_config = VIS_PARAMS[selected_index]

        # 2. Input validation
        if start_year > end_year:
            status_label.value = "Error: Start Year cannot be after End Year."; return
        years_to_process = list(range(start_year, end_year + 1, delta))
        if end_year not in years_to_process: years_to_process.append(end_year)
        
        # 3. Prepare map and geometry
        m = geemap.Map()
        with map_output: display(m)
        
        selected_geometry = FLORIDA_STATE if selected_geo_name == 'Florida' else FLORIDA_COUNTIES.filter(ee.Filter.eq('NAME', selected_geo_name))
        m.centerObject(selected_geometry, 9 if selected_geo_name != 'Florida' else 7)
        m.add_colorbar(vis_config, label=vis_config['label'])
        m.add_layer_control()
        
        # 4a. Statewide Analysis (Choropleth)
        if selected_geo_name == 'Florida':
            output_tabs.layout.display = ''
            
            yearly_fcs = []
            for i, year in enumerate(years_to_process):
                status_label.value = f"Status: Processing {year}..."
                stats_fc = get_yearly_stats_fc(year, month_range, selected_index, FLORIDA_COUNTIES, scale=500)
                yearly_fcs.append(stats_fc)
                
                image_to_display = stats_fc.filter(ee.Filter.neq('mean', None)).reduceToImage(
                    properties=['mean'], reducer=ee.Reducer.first()
                ).rename(selected_index)
                
                is_shown = (i == len(years_to_process) - 1)
                m.addLayer(image_to_display.clip(FLORIDA_STATE), vis_config, f'{selected_index} for {year}', shown=is_shown)
            
            status_label.value = "Status: Aggregating results..."
            all_features = ee.FeatureCollection(yearly_fcs).flatten()
            df = geemap.ee_to_df(all_features)
            
            if df.empty:
                status_label.value = "Status: No data found."; return

            df = df.rename(columns={'NAME': 'County', 'mean': selected_index})
            df = df[['County', 'year', selected_index]].dropna()
            df[selected_index] = df[selected_index].round(4)
            
            with table_output:
                display(df.pivot(index='County', columns='year', values=selected_index))
            
            with rank_output:
                mean_values = df.groupby('County')[selected_index].mean().round(3)
                rank_html = f"""
                <h3>County Rankings (Mean over {start_year}-{end_year})</h3>
                <div style="display: flex; justify-content: space-around;">
                    <div><h4>{vis_config['rank_high']}</h4>{mean_values.nlargest(5).to_frame().to_html(header=False)}</div>
                    <div><h4>{vis_config['rank_low']}</h4>{mean_values.nsmallest(5).to_frame().to_html(header=False)}</div>
                </div>"""
                display(widgets.HTML(rank_html))

        # 4b. Single County Analysis (Raster)
        else:
            output_tabs.layout.display = 'none'
            
            df_data = []
            for i, year in enumerate(years_to_process):
                status_label.value = f"Status: Processing {year}..."
                mean_image = get_mean_image_for_year(year, month_range, selected_geometry, selected_index)
                
                clipped_image = mean_image.clip(selected_geometry)
                is_shown = (i == len(years_to_process) - 1)
                m.addLayer(clipped_image, vis_config, f'{selected_index} for {year}', shown=is_shown)
                
                stat = mean_image.reduceRegion(reducer=ee.Reducer.mean(), geometry=selected_geometry, scale=200, maxPixels=1e9).getInfo()
                if stat and stat.get(selected_index) is not None:
                    df_data.append({'County': selected_geo_name, 'Year': year, selected_index: stat[selected_index]})
            
            if not df_data:
                status_label.value = "Status: No data found."; return
            
            df = pd.DataFrame(df_data)
            df[selected_index] = df[selected_index].round(4)
            with table_output:
                output_tabs.layout.display = ''
                output_tabs.set_title(1, 'Time Series Data')
                output_tabs.selected_index = 0
                display(df)

        status_label.value = "Status: Done."

    except Exception as e:
        import traceback
        error_message = f"An error occurred: {e}"
        status_label.value = error_message
        with map_output:
            clear_output(wait=True)
            display(widgets.HTML(f"<h3>An error prevented the map from loading:</h3><p>{e}</p><pre>{traceback.format_exc()}</pre>"))

# -----------------------------------------------------------------------------
# 7. RUN THE APPLICATION
# -----------------------------------------------------------------------------
run_button.on_click(run_analysis)
display(header, ui_layout)

HTML(value='<h2>Florida Vegetation Index Dashboard</h2>')